# AO3 Web Scraper und Datenextraktor

## Ein suchurl-basiertes Skript zur systematischen Erfassung von Fanfiction-Metadaten und Textkorpora

Dieses Notebook sammelt AO3-Works in zwei klar getrennten Schritten. Zunächst werden über eine AO3 Suche alle relevanten Works identifiziert und deren eindeutige Work-IDs aus den Suchergebnissen extrahiert. In einem zweiten Schritt werden auf Basis dieser Work-IDs gezielt die zugehörigen Metadaten (z. B. Titel, Autor:in, Tags, Ratings, Statistiken) sowie – sofern erforderlich – die Volltexte der Werke abgerufen.

Da Archive of Our Own keine öffentliche Programmierschnittstelle (API) bereitstellt, erfolgt der Zugriff nicht direkt, sondern über die Python-Bibliothek ao3_api, die die AO3-Weboberfläche automatisiert ausliest. Der Einsatz dieser Bibliothek geschieht unter Berücksichtigung der Nutzungsbedingungen von AO3, insbesondere durch begrenzte Abruffrequenzen und einen ressourcenschonenden Zugriff.

### Initiales Setup
Sofern die folgenden Bibliotheken in der Umgebung noch nicht installiert wurden, können diese über nachfolgende Zelle installiert bzw. geupdatet werden.

In [3]:
!uv pip install --upgrade unidecode
!uv pip install --upgrade tqdm
!uv pip install --upgrade requests
!uv pip install --upgrade pandas
!uv pip install --upgrade beautifulsoup4

Resolved 1 package in 49ms
Audited 1 package in 0.82ms
Resolved 2 packages in 77ms
Prepared 1 package in 17ms
Installed 1 package in 100ms
 + tqdm==4.67.1
Resolved 5 packages in 76ms
Audited 5 packages in 1ms
Resolved 6 packages in 115ms
Prepared 2 packages in 13ms
Installed 2 packages in 1.81s
 + pandas==2.3.3
 + pytz==2025.2
Resolved 3 packages in 60ms
Audited 3 packages in 0.73ms


### 1. Sammeln der WorkIds
Die nachfolgende Zelle startet den AO3-ID-Scraper und generiert eine Ausgabedatei mit den einschlägigen work_ids

#### Anpassungsmöglichkeiten:
- AO3 Such-URL: !python Scraper_IDs.py **"https://archiveofourown.org/...ago"**
- Name der Ausgabedatei: **--out_csv Titel.csv**
- Name der log-Datei: **--log_txt Titel.txt**
- Anzahl abzurufender IDs: **--num_to_retrieve 611**
- Header: **--header "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0") ForschungsprojektDH**


In [6]:
#v3
!python HilfsDokumente/AO3-ID-Scraper.py \
"https://archiveofourown.org/works/search?work_search%5Bquery%5D=&work_search%5Btitle%5D=&work_search%5Bcreators%5D=&work_search%5Brevised_at%5D=3+month+ago&work_search%5Bcomplete%5D=&work_search%5Bcrossover%5D=F&work_search%5Bsingle_chapter%5D=0&work_search%5Bsingle_chapter%5D=1&work_search%5Bword_count%5D=&work_search%5Blanguage_id%5D=en&work_search%5Bfandom_names%5D=Harry+Potter+-+J.+K.+Rowling&work_search%5Brating_ids%5D=&work_search%5Barchive_warning_ids%5D%5B%5D=16&work_search%5Bcharacter_names%5D=&work_search%5Brelationship_names%5D=&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=&work_search%5Bcomments_count%5D=&work_search%5Bbookmarks_count%5D=&work_search%5Bsort_column%5D=_score&work_search%5Bsort_direction%5D=desc&commit=Search" \
--out_csv Ausgabedokumente/AO3_HP_IDs_10_2025.csv \
--log_dir Ausgabedokumente \
--log_name AO3_HP_IDs_10_2025.log \
--num_to_retrieve 2200 \
--delay 5 \
--header "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0; UniProjektDH"



Log-Datei gespeichert unter: Ausgabedokumente\AO3_HP_IDs_10_2025.log
===== AO3 SCRAPER LOG =====
Script: ao3_ids
Startzeit: 2026-01-08T20:55:19
Parameter / Aufruf:
  search_url: https://archiveofourown.org/works/search?work_search%5Bquery%5D=&work_search%5Btitle%5D=&work_search%5Bcreators%5D=&work_search%5Brevised_at%5D=3+month+ago&work_search%5Bcomplete%5D=&work_search%5Bcrossover%5D=F&work_search%5Bsingle_chapter%5D=0&work_search%5Bsingle_chapter%5D=1&work_search%5Bword_count%5D=&work_search%5Blanguage_id%5D=en&work_search%5Bfandom_names%5D=Harry+Potter+-+J.+K.+Rowling&work_search%5Brating_ids%5D=&work_search%5Barchive_warning_ids%5D%5B%5D=16&work_search%5Bcharacter_names%5D=&work_search%5Brelationship_names%5D=&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=&work_search%5Bcomments_count%5D=&work_search%5Bbookmarks_count%5D=&work_search%5Bsort_column%5D=_score&work_search%5Bsort_direction%5D=desc&commit=Search
  out_csv: Ausgabedokumente/AO3_HP_I


IDs sammeln:  96%|##################################5 | 2114/2200 [18:42<00:45,  1.88it/s]


### 2. Metadaten und Texte sammeln
Die nachfolgende Zelle startet den AO3-Fanfiction-Scraper und generiert zwei seperate CSV Dateien mit Metadaten bzw. Texten.

AO3 bietet den Vorteil, dass neben den Texten regelmäßig umfangreiche, von den Autor:innen vergebene Tags als Metadaten verfügbar sind. In diesem Skript werden Metadaten und Volltexte getrennt voneinander gescrapet. Diese Trennung erhöht die technische Effizienz und Stabilität des Scraping-Prozesses, da die HTML-Struktur von AO3 bei fehlenden oder variablen Werten andernfalls zu fehlerhaften Zuordnungen von Einträgen führen kann.


#### Anpassungsmöglichkeiten:
- Bezeichnung der Ursprungsdatei, in der die IDs gespeichert sind
- Bezeichnung der Ausgabedateien
- Zeit, die zwischen den Anfragen verstreichen soll; Standard: 5 Sekunden
- Header: --header "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0; UniProjektDH (madh54@gmail.com)"

#### Ergebnis:
- CSV Datei mit work_id und Metadaten
- CSV mit work_id und Texten
- Log Datei mit Abfragedaten
- Error Datei zu fehlgeschlagenen Abfragen

In [7]:
#v3
!python HilfsDokumente/AO3-FanFic-Scraper.py \
--input_csv AO3_HP_IDs_10_2025.csv \
--text_csv AO3_HP_FanFic_10_2015_text.csv \
--meta_csv AO3_HP_FanFic_10_2015_meta.csv \
--error_csv AO3_HP_FanFic_10_2015_errors.csv \
--log_name AO3_HP_FanFic_10_2015.log \
--log_dir Ausgabedokumente \
--delay 5 \
--header "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0; UniProjektDH (madh54@gmail.com)"


[1/2114] Work ID 76158526 verarbeitet.
[2/2114] Work ID 76147721 verarbeitet.
[3/2114] Work ID 75292571 verarbeitet.
[4/2114] Work ID 74128081 verarbeitet.
[5/2114] Work ID 73599016 verarbeitet.
[6/2114] Work ID 73513201 verarbeitet.
[7/2114] Work ID 73512871 verarbeitet.
[8/2114] Work ID 73511111 verarbeitet.
[9/2114] Work ID 73510111 verarbeitet.
[10/2114] Work ID 73509761 verarbeitet.
[11/2114] Work ID 73483876 verarbeitet.
[12/2114] Work ID 73474956 verarbeitet.
[13/2114] Work ID 73474871 verarbeitet.
[14/2114] Work ID 73470161 verarbeitet.
[15/2114] Work ID 73469221 verarbeitet.
[16/2114] Work ID 73467996 verarbeitet.
[17/2114] Work ID 73467906 verarbeitet.
[18/2114] Work ID 73467896 verarbeitet.
[19/2114] Work ID 73467861 verarbeitet.
[20/2114] Work ID 73467631 verarbeitet.
[21/2114] Work ID 73467091 verarbeitet.
[22/2114] Work ID 73467046 verarbeitet.
[23/2114] Work ID 73467011 verarbeitet.
[24/2114] Work ID 73466901 verarbeitet.
[25/2114] Work ID 73465916 verarbeitet.
[26/2114]